# Movie Recommendation System

### Importing important libraries

In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Pre-processing

In [2]:
#loading the data 
df = pd.read_csv(r"movies.csv")

In [3]:
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [4]:
#list of columns
df.columns.values.tolist()

['index',
 'budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'vote_average',
 'vote_count',
 'cast',
 'crew',
 'director']

In [5]:
#no. of rows and columns
df.shape

(4803, 24)

### Feature Selection

In [6]:
#selecting the relevent features for recommendation
selected_features = ['genres','keywords','overview','tagline','cast','director']

In [7]:
selected_features

['genres', 'keywords', 'overview', 'tagline', 'cast', 'director']

In [8]:
#replacing the null values with null values

for feature in selected_features:
    df[feature] = df[feature].fillna('')

In [9]:
# combining all the 6 selected features

combined_features = df['genres']+' '+df['keywords']+' '+df['overview']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [10]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [11]:
#converting the textual data to feature vector

#loading TfidfVectorizer in variable called vectorizer
vectorizer = TfidfVectorizer()

In [12]:
#using vectorizer to fit and transform the data from textual to numerical(feature vectors) and 
#storing it to the variable called feature_vectors

feature_vectors = vectorizer.fit_transform(combined_features)

In [13]:
print(feature_vectors)

  (0, 4288)	0.13213124585063996
  (0, 14180)	0.08418056281586364
  (0, 23134)	0.1498786462809525
  (0, 17784)	0.12285843797047787
  (0, 15569)	0.171366691592621
  (0, 25868)	0.11590870590502833
  (0, 29614)	0.15064979633862852
  (0, 24802)	0.158925784923944
  (0, 23578)	0.16802034155542864
  (0, 30543)	0.15064979633862852
  (0, 30145)	0.18243919685121024
  (0, 23619)	0.11047375838269538
  (0, 19410)	0.0318696249813729
  (0, 30121)	0.07272849549697494
  (0, 9065)	0.14505772980982928
  (0, 5245)	0.16506865163441015
  (0, 1014)	0.12285843797047787
  (0, 1243)	0.05270709034262741
  (0, 21523)	0.1665010643166998
  (0, 1281)	0.03195190270050669
  (0, 19650)	0.1612055740237117
  (0, 10368)	0.13848277654301597
  (0, 2974)	0.09974129089792806
  (0, 27515)	0.14569220419090678
  (0, 2685)	0.09805341831299116
  :	:
  (4802, 27153)	0.05940079357098276
  (4802, 19243)	0.061349419053434766
  (4802, 11941)	0.07829183421073846
  (4802, 29680)	0.07735547381237222
  (4802, 9346)	0.07104321122746965
  (48

#### Cosine Similarity

In [14]:
# gettin the similarity scores using Cosine Similarity

similarity = cosine_similarity(feature_vectors)

In [15]:
similarity

array([[1.        , 0.05083168, 0.0332947 , ..., 0.02749812, 0.0304889 ,
        0.0072518 ],
       [0.05083168, 1.        , 0.04356836, ..., 0.05077045, 0.03100979,
        0.01521198],
       [0.0332947 , 0.04356836, 1.        , ..., 0.02646984, 0.04751623,
        0.01372603],
       ...,
       [0.02749812, 0.05077045, 0.02646984, ..., 1.        , 0.03481447,
        0.03546821],
       [0.0304889 , 0.03100979, 0.04751623, ..., 0.03481447, 1.        ,
        0.03098945],
       [0.0072518 , 0.01521198, 0.01372603, ..., 0.03546821, 0.03098945,
        1.        ]])

In [16]:
similarity.shape

(4803, 4803)

In [17]:
#getting the movie name from user

movie_name = input('Enter the movie name: ')

Enter the movie name: Stuart Little


In [18]:
#creating a list with all the movie names given in the dataset

list_of_all_titles = df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [19]:
# finding the close match for the movie name given by the user 
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

In [20]:
find_close_match

['Stuart Little', 'Stuart Little 2', 'Sugar Hill']

In [21]:
close_match = find_close_match[0]

In [22]:
close_match

'Stuart Little'

In [23]:
# finding the index of the movie with title

index_of_the_movie = df[df.title == close_match]['index'].values[0]
index_of_the_movie

268

In [24]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.016390348226203985), (1, 0.02761450629543333), (2, 0.03165551219011553), (3, 0.018963786249271856), (4, 0.04482002084178288), (5, 0.025023494399864323), (6, 0.03923311604217372), (7, 0.055297892866776936), (8, 0.03899583725023033), (9, 0.04925971115179938), (10, 0.0293136397664935), (11, 0.0373865603154749), (12, 0.016069724770769064), (13, 0.029495660683448083), (14, 0.07275763923019672), (15, 0.09620748636393009), (16, 0.058746321494293596), (17, 0.03707970507026109), (18, 0.03913461289351973), (19, 0.028200042857504617), (20, 0.053770118640545116), (21, 0.026188113839144923), (22, 0.02195718807996725), (23, 0.014766472721821456), (24, 0.015221889509485933), (25, 0.037370786463452876), (26, 0.02473868917805985), (27, 0.04171348248576436), (28, 0.006192906997592593), (29, 0.020767338053952204), (30, 0.03332749781938545), (31, 0.02638954876734325), (32, 0.05675665948401265), (33, 0.06248698268124234), (34, 0.06141936294361795), (35, 0.039235325515240824), (36, 0.0280835670564102

In [25]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(268, 1.0), (221, 0.5209580452807663), (734, 0.14469332967149293), (3059, 0.13781744478018235), (4301, 0.13631604432643382), (144, 0.12342866076916843), (1698, 0.12293524884136688), (3238, 0.11981292221796976), (1112, 0.11942621719791065), (539, 0.1145355348009093), (1117, 0.11439887529690101), (2811, 0.11392503974169294), (1481, 0.11309573785963808), (1686, 0.11202782618662922), (265, 0.10944513314771262), (2044, 0.10906649892980967), (2303, 0.1076454299957975), (707, 0.10697452563382406), (241, 0.10533161779708564), (57, 0.10450373240457991), (1564, 0.10341662834834195), (4575, 0.10291909966834012), (381, 0.10244118185925415), (3583, 0.10142934703938226), (3087, 0.10086052633411022), (45, 0.10038510212140227), (54, 0.09972143226463057), (1999, 0.0996554689142625), (3125, 0.09903361555847291), (1257, 0.09884496212106259), (501, 0.09731961630616745), (1382, 0.09662867988932988), (225, 0.096520786541054), (15, 0.09620748636393009), (2946, 0.0960044845798248), (88, 0.09497007398603462),

In [26]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Stuart Little
2 . Stuart Little 2
3 . The Tale of Despereaux
4 . Teacher's Pet
5 . The Blue Bird
6 . Mr. Peabody & Sherman
7 . The Rugrats Movie
8 . Little Miss Sunshine
9 . Finding Forrester
10 . Titan A.E.
11 . Charlie St. Cloud
12 . Ernest & Celestine
13 . Thunder and the House of Magic
14 . The Borrowers
15 . The Cat in the Hat
16 . The Little Vampire
17 . The Nutcracker
18 . Cheaper by the Dozen 2
19 . Teenage Mutant Ninja Turtles: Out of the Shadows
20 . WALL·E
21 . Legends of the Fall
22 . Napoleon Dynamite
23 . The Nutcracker: The Untold Story
24 . Our Idiot Brother
25 . Nicholas Nickleby
26 . World War Z
27 . The Good Dinosaur
28 . The Adventurer: The Curse of the Midas Box
29 . Eulogy


In [27]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<=30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Stuart Little
Movies suggested for you : 

1 . Stuart Little
2 . Stuart Little 2
3 . The Tale of Despereaux
4 . Teacher's Pet
5 . The Blue Bird
6 . Mr. Peabody & Sherman
7 . The Rugrats Movie
8 . Little Miss Sunshine
9 . Finding Forrester
10 . Titan A.E.
11 . Charlie St. Cloud
12 . Ernest & Celestine
13 . Thunder and the House of Magic
14 . The Borrowers
15 . The Cat in the Hat
16 . The Little Vampire
17 . The Nutcracker
18 . Cheaper by the Dozen 2
19 . Teenage Mutant Ninja Turtles: Out of the Shadows
20 . WALL·E
21 . Legends of the Fall
22 . Napoleon Dynamite
23 . The Nutcracker: The Untold Story
24 . Our Idiot Brother
25 . Nicholas Nickleby
26 . World War Z
27 . The Good Dinosaur
28 . The Adventurer: The Curse of the Midas Box
29 . Eulogy
30 . Fantastic Mr. Fox
